In [ ]:
%%bash
DATADIR=/blob/v-jinhzh/data/kddcup/raw
BINDIR=/blob/v-jinhzh/data/kddcup/bindata
for subset in train valid test; do
    python molecule/canonicalize.py $DATADIR/${subset}.x --workers 30
done 
for subset in train valid test; do
    python molecule/tokenize_re.py $DATADIR/${subset}.x.can \
        --workers 30 --output-fn $DATADIR/${subset}.x.bpe 
done 

tgtdir=${BINDIR}/input0
mkdir -p $tgtdir
fairseq-preprocess \
    --only-source \
    --trainpref $DATADIR/train.x.bpe \
    --validpref $DATADIR/valid.x.bpe \
    --testpref $DATADIR/test.x.bpe \
    --destdir $tgtdir \
    --workers 30 \
    --molecule

In [ ]:
%%bash
# python convert_reg2cls.py
DATADIR=/blob/v-jinhzh/data/kddcup/raw
BINDIR=/blob/v-jinhzh/data/kddcup/bindata
tgtdir=${BINDIR}/label_cls_${numel}


fairseq-preprocess \
    --only-source  \
    --trainpref $DATADIR/train.${numel}.y \
    --validpref $DATADIR/valid.${numel}.y \
    --destdir $tgtdir \
    --workers 30 \
    --srcdict ${DATADIR}/dict.${numel}.txt

tgtdir=${BINDIR}/label_reg
mkdir -p $tgtdir
for subset in train valid; do 
    cp $DATADIR/${subset}.y $tgtdir/${subset}.y
done 


In [ ]:
%%bash 
DATADIR=/blob/v-jinhzh/data/kddcup/rawdev
rm -rf $DATADIR
mkdir -p $DATADIR
BINDIR=/blob/v-jinhzh/data/kddcup/bindatadev
rm -rf $BINDIR
mkdir -p $BINDIR
originaldata=/blob/v-jinhzh/data/kddcup/raw
for subset in train valid; do
    cp $originaldata/${subset}.x.bpe $DATADIR
    cp $originaldata/${subset}.17144.y $DATADIR
    cp $originaldata/${subset}.y $DATADIR
done 
cp $originaldata/test.x.bpe $DATADIR

cat $DATADIR/valid.x.bpe >> $DATADIR/train.x.bpe
cat $DATADIR/valid.17144.y >> $DATADIR/train.17144.y
cat $DATADIR/valid.y >> $DATADIR/train.y

tgtdir=${BINDIR}/input0
mkdir -p $tgtdir
fairseq-preprocess \
    --only-source \
    --trainpref $DATADIR/train.x.bpe \
    --validpref $DATADIR/valid.x.bpe \
    --testpref $DATADIR/test.x.bpe \
    --destdir $tgtdir \
    --workers 30 \
    --molecule \
    --srcdict /blob/v-jinhzh/data/kddcup/bindata/input0/dict.txt

numel=17144
tgtdir=${BINDIR}/label_cls_${numel}
fairseq-preprocess \
    --only-source  \
    --trainpref $DATADIR/train.${numel}.y \
    --validpref $DATADIR/valid.${numel}.y \
    --destdir $tgtdir \
    --workers 30 \
    --srcdict /blob/v-jinhzh/data/kddcup/raw/dict.17144.txt 

tgtdir=${BINDIR}/label_reg
mkdir -p $tgtdir
for subset in train valid; do 
    cp $DATADIR/${subset}.y $tgtdir/${subset}.y
done